# Продвинутое машинное обучение: Домашнее задание 3

Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются.

В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса. Как известно, в рассказе The Adventure of the Dancing Men великий сыщик расшифровал загадочные письмена, которые выглядели примерно так:

![](img/dancing_men.png)

Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.

В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам “Война и мир” по-русски и по-английски:


## 1. Baseline

Реализуйте базовый частотный метод по Шерлоку Холмсу:

- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.

In [31]:
import re
import random
from typing import List, Dict, Callable
from collections import Counter

import tqdm
import numpy as np


def read_texts(data_path: str) -> List[str]:
    with open(data_path, "r") as input_stream:
        texts = input_stream.read().splitlines()
    
    return texts


def preprocess_rus(text: str):
    text = re.sub(r'[^а-ё ]+', '', text.lower())
    text = re.sub(' +', ' ', text)
    return text


def preprocess_eng(text: str):
    text = re.sub(r'[^a-z ]+', '', text.lower())
    text = re.sub(' +', ' ', text)
    return text


def preprocess_texts(
    texts: List[str], first_sentence: str = '', last_sentence: str = '', alphabet: str = 'rus'
) -> str:

    if alphabet == 'rus':
        texts = [preprocess_rus(text) for text in texts]
    elif alphabet == 'eng':
        texts = [preprocess_eng(text) for text in texts]
    else:
        raise NotImplementedError()

    preprocessed = [text for text in texts if text]

    if first_sentence:
        index_first = texts.index(first_sentence)
        texts = texts[index_first:]

    if last_sentence:
        index_end = texts.index(last_sentence)
        texts = texts[:index_end]
    
    texts = ' '.join(texts)

    return texts

In [2]:
data_path = "data/AnnaKarenina.txt"
first_sentence = "все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива посвоему"
last_sentence = "примечания"

texts = read_texts(data_path)
corpus = preprocess_texts(texts, first_sentence, last_sentence, 'rus')

In [3]:
# data_path = "data/WarAndPeace.txt"
# first_sentence = "часть первая"
# last_sentence = ""

# texts = read_texts(data_path)
# corpus = preprocess_texts(texts, first_sentence, last_sentence, 'rus')

In [4]:
# data_path = "data/WarAndPeaceEng.txt"
# first_sentence = "well prince so genoa and lucca are now just family estates of the"
# last_sentence = "end of the project gutenberg ebook of war and peace by leo tolstoy"

# texts = read_texts(data_path)
# corpus = preprocess_texts(texts, first_sentence, last_sentence, 'eng')

In [5]:
def get_sorted_chars(text: str) -> List[chr]:
    """Возвращает список биграм в порядке уменьшения частоты встречаемости в тексте text"""
    
    sorted_chars = [item[0] for item in Counter(text).most_common()]
    
    return sorted_chars


def encode_chars(text: str, encoding: Dict[chr, chr]) -> str:
    """Кодирует текст text при помощи замены символов encoding"""
    
    encoded_text = ''.join(map(lambda x: encoding.get(x, ' '), text))
    return encoded_text


def encode_decode_chars(text: str, corpus: str) -> Dict[str, str]:
    """Кодирует текст text, а затем декодирует при помощи корпуса corpus, используя частотный метод по символам"""
    
    # посчитаем частоты n-грамм в корпусе
    sorted_corpus_chars = get_sorted_chars(corpus)
    
    # случайным образом переставим буквы в алфавите
    shuffling = sorted_corpus_chars.copy()
    random.shuffle(shuffling)
    encoding = dict(zip(sorted_corpus_chars, shuffling))
    
    # закодируем текст случайной перестановкой букв
    encoded_text = encode_chars(text, encoding)
    
    # посчитаем частоты n-грамм в закодированном тексте
    sorted_encoded_text_chars = get_sorted_chars(encoded_text)
    
    # декодируем текст
    inverse_encoding = dict(zip(sorted_encoded_text_chars, sorted_corpus_chars))
    decoded_text = encode_chars(encoded_text, inverse_encoding)
    
    # результат в удобном виде
    result = {
        "original_text": text,
        "encoded_text": encoded_text,
        "decoded_text": decoded_text,
    }
    
    return result

In [6]:
text = """
    Есть люди, которые, встречая своего счастливого в чем бы то ни было соперника,
    готовы сейчас же отвернуться от всего хорошего, что есть в нем, и видеть в нем одно дурное;
    есть люди, которые, напротив, более всего желают найти в этом счастливом сопернике те качества,
    которыми он победил их, и ищут в нем со щемящею болью в сердце одного хорошего.
"""

text = preprocess_rus(text.strip())

In [7]:
decode_chars_result = encode_decode_chars(text, corpus)
decode_chars_result

{'original_text': 'есть люди которые встречая своего счастливого в чем бы то ни было соперника готовы сейчас же отвернуться от всего хорошего что есть в нем и видеть в нем одно дурное есть люди которые напротив более всего желают найти в этом счастливом сопернике те качества которыми он победил их и ищут в нем со щемящею болью в сердце одного хорошего',
 'encoded_text': 'шбсрйжчёяйьцсцтишйыбстшгюмйбыцшэцйбгюбсжяыцэцйыйгшъй ийсцйеяй ижцйбцуштеяьюйэцсцыийбшнгюбйпшйцсыштеосрбмйцсйыбшэцйфцтцлшэцйгсцйшбсрйыйешъйяйыяёшсрйыйешъйцёецйёотецшйшбсрйжчёяйьцсцтишйеюутцсяый цжшшйыбшэцйпшжючсйеюнсяйыйхсцъйбгюбсжяыцъйбцуштеяьшйсшйьюгшбсыюйьцсцтиъяйцейуц шёяжйяфйяйядосйыйешъйбцйдшъмдшчй цжрчйыйбштёзшйцёецэцйфцтцлшэц',
 'decoded_text': 'енап рыкт яоаолье иналеувч ниоедо нувнартиодо и уем гь ао ст гьро нобелстяв доаоиь нешувн хе оаиелсзапнч оа инедо жолоюедо уао енап и сем т иткеап и сем оксо кзлсое енап рыкт яоаолье свблоати горее инедо хервыа свшат и эаом нувнартиом нобелстяе ае явуенаив яоаольмт ос бо

In [8]:
def accuracy_score(text1, text2):
    """Процент корректно расшифрованных букв"""
    
    correct = sum([a == b for a, b in zip(text1, text2)])
    score = correct / len(text1)
    
    return score

In [9]:
accuracy_score(
    decode_chars_result["original_text"],
    decode_chars_result["decoded_text"]
)

0.4221556886227545

## 2. Bigrams

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:

- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм


In [10]:
def get_bigrams(text: str) -> List[str]:
    """Возвращает список биграм в произвольном порядке"""
    
    return [text[i] + text[i + 1] for i in range(0, len(text) - 1, 2)]


def get_sorted_bigrams(text: str) -> List[chr]:
    """Возвращает список биграм в порядке уменьшения частоты встречаемости в тексте text"""
    
    bigrams = get_bigrams(text)
    sorted_bigrams = [item[0] for item in Counter(bigrams).most_common()]
    
    return sorted_bigrams


def encode_bigrams(text: str, encoding: Dict[chr, chr]) -> str:
    """Кодирует текст text при помощи замены биграм encoding"""
    
    bigrams = get_bigrams(text)
    encoded_text = ''.join(map(lambda x: encoding.get(x, ' '), bigrams))
    
    return encoded_text


def encode_decode_bigrams(text: str, corpus: str) -> Dict[str, str]:
    """Кодирует текст text, а затем декодирует при помощи корпуса corpus, используя частотный метод по биграмам"""
    
    # посчитаем частоты биграм в корпусе
    sorted_corpus_bigrams = get_sorted_bigrams(corpus)
    
    # случайным образом переставим биграмы
    shuffling = sorted_corpus_bigrams.copy()
    random.shuffle(shuffling)
    encoding = dict(zip(sorted_corpus_bigrams, shuffling))
    
    # закодируем текст случайной перестановкой биграм
    encoded_text = encode_bigrams(text, encoding)
    
    # посчитаем частоты биграм в закодированном тексте
    sorted_encoded_text_bigrams = get_sorted_bigrams(encoded_text)
    
    # декодируем текст
    inverse_encoding = dict(zip(sorted_encoded_text_bigrams, sorted_corpus_bigrams))
    decoded_text = encode_bigrams(encoded_text, inverse_encoding)
    
    # результат в удобном виде
    result = {
        "original_text": text,
        "encoded_text": encoded_text,
        "decoded_text": decoded_text,
    }
    
    return result

In [11]:
decode_bigrams_result = encode_decode_bigrams(text, corpus)
decode_bigrams_result

{'original_text': 'есть люди которые встречая своего счастливого в чем бы то ни было соперника готовы сейчас же отвернуться от всего хорошего что есть в нем и видеть в нем одно дурное есть люди которые напротив более всего желают найти в этом счастливом сопернике те качества которыми он победил их и ищут в нем со щемящею болью в сердце одного хорошего',
 'encoded_text': ' слзкцьэтэтмдыудбкурыжлюбгдз члднд гуубиьиоунд ттудцхмдшндтстзоонд ькфюрхгпюусдыщодзчмзтыщмшьцянязвълзмяьцсшурлдндзърыдбусёкдыикычпо тшндцтэрдвчлзнрчсаеьцчандбъюрнжикычпохсея япьншижчсфорыэ  тйдчобкурлдндмшзьоччсхуэ нреидыдц гуубиьиюкдзосязтсбцдшбкцытуычер япьншаютэфувегруе щзрадтэсььхнрчсаедзндбыорбырюйджлрю тгодмельцчаоундзърыдбус',
 'decoded_text': ' чтотьпре оса ло п оелн  ми ор ио ално к  оно  нгоя вапоо каза ео оллеков ернаа лии ретатела сом тскток  сет о ио ниь  днаога овстл  ниля е вичттоне вен сй о декоановстл т ривом  бсе вы ь от ну  у п о ио ладаак вазотнебыа я ално к  веи  з ткаатпо пингостжевом  боде емит 

In [12]:
accuracy_score(
    decode_bigrams_result["original_text"],
    decode_bigrams_result["decoded_text"]
)

0.17365269461077845

## 3. MCMC on bigrams

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:

- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.

In [40]:
def get_bigrams_freq(text: str) -> Dict[str, float]:
    bigrams = get_bigrams(text)
    bigrams_freq = dict(Counter(bigrams).most_common())
    bigrams_freq = {k: v / len(bigrams) for k, v in bigrams_freq.items()}
    
    return bigrams_freq


def get_log_proba(decoded_text: str, corpus_bigrams_freq: Dict[str, int]) -> float:
    min_freq = min(corpus_bigrams_freq.values())
    log_proba = 0
    
    for i in range(0, len(decoded_text) - 1):
        bigram = text[i] + decoded_text[i + 1]
        proba = corpus_bigrams_freq.get(bigram, min_freq)
        log_proba += np.log(proba)
    
    return log_proba


def encode_decode_mmc(text: str, corpus: str, num_trials: int = 1000, num_steps: int = 10) -> Dict[str, str]:
    # получим отсортированные по убыванию частоты биграмы корпуса
    sorted_corpus_bigrams = get_sorted_bigrams(corpus)
    
    # случайным образом переставим биграмы
    shuffling = sorted_corpus_bigrams.copy()
    random.shuffle(shuffling)
    encoding = dict(zip(sorted_corpus_bigrams, shuffling))
    
    # закодируем текст случайной перестановкой биграм
    encoded_text = encode_bigrams(text, encoding)
    
    # получим отсортированные по убыванию частоты биграмы закодированного текста
    sorted_encoded_text_bigrams = get_sorted_bigrams(encoded_text)
    
    # получим частоты биграм в корпусе
    corpus_bigrams_freq = get_bigrams_freq(corpus)
    
    best_log_proba = float("-inf")
    best_inverse_encoding = None
    
    for trial in tqdm.tqdm(range(num_trials)):

        inverse_encoding = dict(zip(sorted_encoded_text_bigrams, sorted_corpus_bigrams))
        decoded_text = encode_bigrams(encoded_text, inverse_encoding)
        log_proba = get_log_proba(decoded_text, corpus_bigrams_freq)
        
        for step in range(num_steps):
            new_text_bigrams = sorted_encoded_text_bigrams.copy()
            
            i, j = np.random.choice(len(sorted_encoded_text_bigrams), size=2, replace=False)
            new_text_bigrams[i], new_text_bigrams[j] = new_text_bigrams[j], new_text_bigrams[i]
            
            new_inverse_encoding = dict(zip(new_text_bigrams, sorted_corpus_bigrams))
            new_decoded_text = encode_bigrams(encoded_text, new_inverse_encoding)
            new_log_proba = get_log_proba(new_decoded_text, corpus_bigrams_freq)
            
            p = np.exp(new_log_proba - log_proba)
            
            if p > random.random():
                log_proba = new_log_proba
                inverse_encoding = new_inverse_encoding
            
        if log_proba > best_log_proba:
            max_log_proba = log_proba
            best_inverse_encoding = inverse_encoding
    
    decoded = encode_bigrams(encoded_text, best_inverse_encoding)
    
    return decoded

In [41]:
encode_decode_mmc(text, corpus, num_trials=5000, num_steps=30)

100%|██████████| 5000/5000 [01:07<00:00, 74.14it/s]


' чтотьпре оса ло п оелн  ми ор ио ално к  оно  нгоя вапоо каза ео оллеков ернаа лии ретатела сом тскток  сет о ио ниен днаога овстл  ниля е вичттоне вь  сй о декоановстл т ривом  бсе вы енот ну  у п о ио ладаак вазотнебыа я ално к  веи  з ткаатпо пингостжевом  боде емит ртиесегдое  ласне вь и о рольрорау всра ню обче сй оно ниен дна'

## 4. Predict

Расшифруйте сообщение:

In [45]:
text = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"# text2 = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"

In [47]:
encode_decode_mmc(text1, corpus, num_trials=5000, num_steps=30)

## 5*. Trigrams

Бонус: а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

## 6*. Assume
Какие вы можете придумать применения для этой модели? Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).